K-means is a very simple but powerful clustering algorithm. In its native form, the algorithm is very easy to understand:
1. Initialize randomly k points (called centroids  or centers) in the same multidimensional space of the input
2. Assign each of the input points to its closest center
3. Update each center with the mean of the points assigned to it.
4. Repeat 2-3 until convergence (i.e. the assignment to the clusters remains unchanged)

In this tutorial, we will provide the implementation of a clustering algorithm that is inspired by k-means but it can be implemented inside a gradient descent scheme, allowing us to incorporate this algorithm inside a general TensorFlow graph.

The method works as following:
1. Initialize randomly k variables (the centers)
2. Assign each of the batch points to its closest center
3. Minimize the average distance between each point and its assigned center

This method is teoretically sound, since it can be shown that a gradient descent scheme performs updates of the centers that are an approximation of the mean of the assigned points.

For this implementation, we will use `sklearn` library to automatically construct a dataset for us.

In [1]:
from sklearn import datasets as dt
import numpy as np
import tensorflow as tf

Let `D` be the dimension of the input space, `N` the number of the input points and `K` the number of centers.

In [2]:
D = 2
N = 100
K = 10

Let's generate the dataset and convert the input into a tensor.

In [3]:
inputs, labels = dt.make_blobs(centers=K, n_features=D, n_samples=N)
X = tf.to_float(inputs)

Let's create the centers' `Variable`s. 

In [4]:
M = tf.Variable(initial_value=tf.truncated_normal(shape=[K, D]), dtype=tf.float32)

Now we need to compute the distance between each element of `X` and each element of `M`. To this end we will expand the dimensions of both the tensors and then we evaluate the distance by: _(i)_ computing the component wise difference, _(ii)_ computing the square of these differences, and _(iii)_ reducing the component dimension by summing up squared differences

In [5]:
Xex = tf.tile(tf.expand_dims(X, axis=1), [1,K,1])
Mex = tf.tile(tf.expand_dims(M, axis=0), [N,1,1])
dist = tf.reduce_sum(tf.square(Xex - Mex), axis=2)

Finally, we will compute the cluster assignment (` argmin` over distances) and convert it into a one-hot representation `R` (1 for closest cluster, 0 elsewhere)

In [6]:
y = tf.argmin(dist, axis=1)
R = tf.one_hot(y, depth=K)

We are ready to define our objective function. Straightforwarly, it is the sum of all the distances of each data point to its assigned cluster

In [7]:
cost = tf.reduce_sum(tf.multiply(R, dist))

Now, we create the training operation using ` AdamOptimizer` for the minimization of `cost`. Then, we will train the model using a fixed number of iterations (just standard Tensorflow learning code).

In [8]:
train_op = tf.train.AdamOptimizer(0.001).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        _ = sess.run((train_op))
    Y = sess.run(y)

Finally, let's plot the learned clusters.

In [9]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
for i in range(K):
    plt.scatter(inputs[Y==i, 0], inputs[Y==i, 1])
plt.show()